# 1 Extraction of Entities and triples

## 1.1 Fetching Articles' (MetaData + Abstract) in OpenAlex --> to csv


In [1]:
%pip install requests
%pip install pandas
%pip install time
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\kacel\Bureau\Bachelor Système d'information\3ème Année\Intelligence_Artificielle\TP_FINAL_logistic_regression\ia_projet_final\venv\Scripts\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\kacel\Bureau\Bachelor Système d'information\3ème Année\Intelligence_Artificielle\TP_FINAL_logistic_regression\ia_projet_final\venv\Scripts\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement time (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\kacel\Bureau\Bachelor Système d'information\3ème Année\Intelligence_Artificielle\TP_FINAL_logistic_regression\ia_projet_final\venv\Scripts\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for time



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\kacel\Bureau\Bachelor Système d'information\3ème Année\Intelligence_Artificielle\TP_FINAL_logistic_regression\ia_projet_final\venv\Scripts\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import time
import requests
import numpy as np
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)


NumPy version: 2.2.1
Pandas version: 2.2.3


In [3]:
# Generic function to retrieve paginated items from OpenAlex
def get_all_items(url, params, item_label):
    session = requests.Session()  # Reuse session for better performance
    all_items = []
    cursor = "*"  # OpenAlex uses cursor-based pagination, "*" means start
    while cursor:
        params["cursor"] = cursor  # Update request with the current cursor
        try:
            response = session.get(url, params=params, timeout=10)
            response.raise_for_status()  # Raise error for HTTP errors
            data = response.json()  # Convert response to JSON
            items = data.get("results", [])  # Get 'results' list from response
            all_items.extend(items)  # Add to full list
            cursor = data.get("meta", {}).get("next_cursor")  # Get next cursor
            print(f"Fetched {len(items)} {item_label}. Next cursor: {cursor}")
            time.sleep(0.1)  # Avoid hitting rate limits
        except Exception as e:
            print(f"Error fetching {item_label}: {e}")
            break
    print(f"Total fetched {item_label}: {len(all_items)}")
    return all_items

# Function to retrieve all subtopics under "Quantum Computing" using concept ID
def get_quantum_computing_subtree(ancestor_concept_id="C58053490"):
    url = "https://api.openalex.org/concepts"
    params = {
        "filter": f"ancestors.id:{ancestor_concept_id}",  # Get all children
        "per-page": 200,
        "select": "id"
    }
    all_concepts_data = get_all_items(url, params, "concepts")
    all_concept_ids = [concept['id'] for concept in all_concepts_data]
    return all_concept_ids

# Function to get all works (papers) tagged with any of the quantum computing concepts
def get_quantum_computing_works(subtree_concept_ids):
    url = "https://api.openalex.org/works"
    concepts_filter = "|".join(subtree_concept_ids)  # OR filter for concepts
    params = {
        "filter": f"concepts.id:{concepts_filter}",
        "per-page": 200
    }
    all_works = get_all_items(url, params, "works")
    return all_works

# Function to export the works into a CSV file, now including abstracts
def export_works_to_csv(works, filename="quantum_computing_subtree_papers.csv"):
    formatted_works = []
    for work in works:
        paper_id = work.get("id", "N/A").replace("https://openalex.org/", "")
        title = work.get("display_name", "N/A")
        publication_year = work.get("publication_year", "N/A")
        journal = work.get("host_venue", {}).get("display_name", "N/A")
        authors = ", ".join(
            auth.get("author", {}).get("display_name", "N/A")
            for auth in work.get("authorships", [])
        )
        citation_count = work.get("cited_by_count", "N/A")

        # ➕ NEW: Get the abstract (OpenAlex stores it in inverted index format)
        abstract_inverted_index = work.get("abstract_inverted_index", None)
        if abstract_inverted_index:
            # Reconstruct abstract from the inverted index
            position_map = []
            for word, positions in abstract_inverted_index.items():
                for pos in positions:
                    position_map.append((pos, word))
            abstract = " ".join(word for _, word in sorted(position_map))
        else:
            abstract = "N/A"

        # Extract concepts (id|name|score)
        # Keep only the concept names
        concept_names = "; ".join([
            concept.get("display_name", "N/A")
            for concept in work.get("concepts", [])
        ])


        # Append all info to our list
        formatted_works.append({
            "paper_id": paper_id,
            "title": title,
            "abstract": abstract,
            "publication_year": publication_year,
            "journal": journal,
            "authors": authors,
            "citation_count": citation_count,
            "concept_names": concept_names  # only names!
        })


    # Optional cleaning (removes any $ symbols)
    for row in formatted_works:
        for key, value in row.items():
            if isinstance(value, str):
                row[key] = value.replace("$", "")

    df = pd.DataFrame(formatted_works)
    df.to_csv(filename, index=False, encoding="utf-8")
    print(f"Data exported to {filename}")

# Main execution block
if __name__ == "__main__":
    quantum_computing_ancestor_id = "C58053490"  # OpenAlex ID for Quantum Computing
    subtree_concept_ids = get_quantum_computing_subtree(quantum_computing_ancestor_id)
    print(f"Fetched {len(subtree_concept_ids)} concept IDs in the Quantum Computing subtree.")

    if subtree_concept_ids:
        works = get_quantum_computing_works(subtree_concept_ids)
        export_works_to_csv(works, filename="quantum_computing_subtree_papers.csv")
    else:
        print("No concepts found in the Quantum Computing subtree. Cannot fetch works.")


Fetched 53 concepts. Next cursor: IlsyNjYsICdodHRwczovL29wZW5hbGV4Lm9yZy9DMTIyODA2MjU3J10i
Fetched 0 concepts. Next cursor: None
Total fetched concepts: 53
Fetched 53 concept IDs in the Quantum Computing subtree.
Fetched 200 works. Next cursor: IlsxMDAuMCwgMTAzOSwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1cxOTkzMjQ4Nzc4J10i
Fetched 200 works. Next cursor: IlsxMDAuMCwgNjUwLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5NjMzNjA1NDEnXSI=
Fetched 200 works. Next cursor: IlsxMDAuMCwgNTA0LCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI1OTM0NTc5MTgnXSI=
Fetched 200 works. Next cursor: IlsxMDAuMCwgNDA2LCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzE5ODI5MjM4OTAnXSI=
Fetched 200 works. Next cursor: IlsxMDAuMCwgMzI4LCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzI5Njc1MzI4MzMnXSI=
Fetched 200 works. Next cursor: IlsxMDAuMCwgMjY5LCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzIwNjI3Mzg4NjYnXSI=
Fetched 200 works. Next cursor: IlsxMDAuMCwgMjIzLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzMxMDQxNTc3MDUnXSI=
Fetched 200 works. Next cursor: IlsxMDAuMCwgMTcyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcv

In [4]:
df = pd.read_csv("quantum_computing_subtree_papers.csv")
df.head()

,paper_id,title,abstract,publication_year,journal,authors,citation_count,concept_names
0,W2148066840,Phase annealing in SHELX-90: direct methods fo...,A number of extensions to the multisolution ap...,1990,NaN,George M. Sheldrick,16025,Simulated annealing; Annealing (glass); Comput...
1,W3037737784,Quantum entanglement,All our former experience with application of ...,2009,NaN,"Ryszard Horodecki, Paweł Horodecki, Michał Hor...",8757,Quantum entanglement; Quantum discord; Physics...
2,W2011208902,Entanglement of Formation of an Arbitrary Stat...,The entanglement of a pure state of a pair of ...,1998,NaN,William K. Wootters,7715,Quantum entanglement; W state; Quantum mechani...
3,W2168676717,Algorithms for quantum computation: discrete l...,A computer is generally considered to be a uni...,2002,NaN,Peter W. Shor,6960,Discrete logarithm; Quantum computer; Post-qua...
4,W2990961515,Quantum Computation and Quantum Information,One of the most cited books in physics of all ...,2012,NaN,"Michael A. Nielsen, Isaac L. Chuang",6882,Quantum computer; Quantum information; Quantum...


## 1.2 Entity extraction and relation extraction using Spacy

In [5]:
# 1. Upgrade pip
!pip install --upgrade pip
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_sm --direct
!python -m spacy info
!python -m spacy validate
!python -m spacy link en_core_web_sm en_core_web_sm --force

%pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------- ----- 1.6/1.8 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 6.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --- ------------------------------------ 1.0/12.8 MB 7.1 MB/s eta 0:00:02
     ---------- ----------------------------- 3.4/12.8 MB 10.1 MB/s eta 0:00:01
     ------------------ --------------------- 6.0/12.8 MB 10.8 MB/s eta 0:00:01
     --------------------------- ------------ 8.7/12.8 MB 11.2 MB/s eta 0:00:01
     --------------------------------- ----- 11.0/12.8 MB 11.3 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 11.3 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 10.3 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Defaulting to user installation because normal site-packages is not writeable


ERROR: Invalid wheel filename (invalid version): '-en_core_web_sm-py3-none-any'



============================== Info about spaCy ==============================

spaCy version    3.8.5                         
Location         C:\Users\kacel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\spacy
Platform         Windows-11-10.0.26100-SP0     
Python version   3.12.10                       
Pipelines        en_core_web_sm (3.8.0)        


| Loading compatibility table...
/ Loading compatibility table...
- Loading compatibility table...
\ Loading compatibility table...
| Loading compatibility table...
/ Loading compatibility table...
- Loading compatibility table...
[+] Loaded compatibility table

================= Installed pipeline packages (spaCy v3.8.5) =================
[i] spaCy installation:
C:\Users\kacel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\spacy

NAME             SPACY            VERSION          

Usage: python -m spacy link [OPTIONS] ARGS KWARGS
Try 'python -m spacy link --help' for help.
+- Error ---------------------------------------------------------------------+
| No such option: --force                                                     |
+-----------------------------------------------------------------------------+



  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\kacel\Bureau\Bachelor Système d'information\3ème Année\Intelligence_Artificielle\TP_FINAL_logistic_regression\ia_projet_final\venv\Scripts\python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import spacy
import en_core_web_sm

# Load SpaCy model
nlp = en_core_web_sm.load()

# Load dataset
df = pd.read_csv("quantum_computing_subtree_papers.csv")                # Force quoting for all fields

# Select only relevant ones
df = df[["paper_id", "title", "abstract", "publication_year", "authors", "concept_names"]]

# Take only the first 100 rows
df = df.head(100)

results = []

# Function to clean the abstract
def clean_abstract(abstract):
    doc = nlp(abstract)
    # Remove stopwords, punctuation, and lemmatize the tokens
    cleaned = ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])
    return cleaned

# Process each row
for index, row in df.iterrows():
    abstract = row["abstract"]
    concepts = row["concept_names"]

    if pd.isna(abstract):
        continue

    # Clean the abstract
    cleaned_abstract = clean_abstract(abstract)

    doc = nlp(cleaned_abstract)
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    results.append({
        "paper_id": row["paper_id"],
        "title": row["title"],
        "abstract": cleaned_abstract,  # Use the cleaned abstract here
        "publication_year": row["publication_year"],
        "authors": row["authors"],
        "concept_names": concepts,
        "entities": entities
    })

processed_df = pd.DataFrame(results)



In [36]:
processed_df["abstract"]

0     number extension multisolution approach crysta...
1     experience application quantum theory \it pred...
2     entanglement pure state pair quantum system de...
3     computer generally consider universal computat...
4     cited book physics time Quantum Computation Qu...
                            ...                        
70    detailed account way quantum computer scheme q...
71    introduce concept quantum random walk quantum ...
72    dictum ` ` information physical indicate under...
73    Quantum cryptography arguably fast grow area q...
74    Abstract counter intuitive aspect quantum phys...
Name: abstract, Length: 75, dtype: object

In [8]:
# Load model
import spacy
nlp = spacy.load("en_core_web_sm")

# Add sentencizer if not already present
if "parser" not in nlp.pipe_names and "sentencizer" not in nlp.pipe_names:
    nlp.add_pipe("sentencizer")


In [9]:
def extract_triples(text):
    doc = nlp(text)
    triples = []
    for sent in doc.sents:
        subject = ''
        verb = ''
        obj = ''
        for token in sent:
            if "subj" in token.dep_:
                subject = token.text
                verb = token.head.lemma_
            if "obj" in token.dep_ and subject:
                obj = token.text
                triples.append((subject, verb, obj))
                subject = ''
                verb = ''
                obj = ''
    return triples

processed_df["triples"] = processed_df["abstract"].apply(extract_triples)


In [37]:
processed_df["triples"]

0     [(relation, play, phase), (idea, incorporate, ...
1     [(entanglement, recognize, Rosen), (schr\"odin...
2                           [(formation, pair, matrix)]
3     [(mechanic, take, researcher), (basis, propose...
4     [(edition, include, author), (textbook, descri...
                            ...                        
70    [(computation, consist, state), (universality,...
71                                                   []
72                        [(Colloquium, discuss, area)]
73    [(field, start, base), (network, overcome, res...
74    [(complementarity, define, processing), (trap,...
Name: triples, Length: 75, dtype: object

In [22]:
%pip install rdflib
%pip install rdflib[extra]


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\kacel\Bureau\Bachelor Système d'information\3ème Année\Intelligence_Artificielle\TP_FINAL_logistic_regression\ia_projet_final\venv\Scripts\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\kacel\Bureau\Bachelor Système d'information\3ème Année\Intelligence_Artificielle\TP_FINAL_logistic_regression\ia_projet_final\venv\Scripts\python.exe -m pip install --upgrade pip


In [42]:
import rdflib
import pandas as pd

g = rdflib.Graph()

g.parse("PhySci/physci.ttl", format="ttl")

def map_to_ontology(triple):
    subj, pred, obj = triple
    subj_uri = rdflib.URIRef(f"https://w3id.org/SKGO/phy#{subj}")
    obj_uri = rdflib.URIRef(f"https://w3id.org/SKGO/phy#{obj}")

    enhanced_triple = (subj, pred, obj)

    if isinstance(obj, str) and re.match(r"\d{2}-\d{2}-\d{4}", obj):
        obj = fix_date_format(obj)

    subj_query = f"""
    ASK WHERE {{
        <{subj_uri}> rdf:type ?type .
    }}
    """

    result = g.query(subj_query)
    for row in result:
        if row[0]:
            print(f"Subject {subj} found in ontology as type: {row[0]}")
            enhanced_triple = (subj, pred, obj)  # Add more context if needed

    obj_query = f"""
    ASK WHERE {{
        <{obj_uri}> rdf:type ?type .
    }}
    """

    # Run the SPARQL query for the object
    result = g.query(obj_query)
    for row in result:
        if row[0]:
            print(f"Object {obj} found in ontology as type: {row[0]}")
            enhanced_triple = (subj, pred, obj)  # Add more context if needed

    return enhanced_triple



Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_xsd_date at 0x000001E24BD04F40>
Traceback (most recent call last):
  File "C:\Users\kacel\Bureau\Bachelor Système d'information\3ème Année\Intelligence_Artificielle\TP_FINAL_logistic_regression\ia_projet_final\venv\Lib\site-packages\rdflib\term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\kacel\Bureau\Bachelor Système d'information\3ème Année\Intelligence_Artificielle\TP_FINAL_logistic_regression\ia_projet_final\venv\Lib\site-packages\rdflib\xsd_datetime.py", line 593, in parse_xsd_date
    return parse_date(date_string if not minus else ("-" + date_string))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: Invalid isoformat string: '01-07-2019'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema

In [ ]:
processed_df["enhanced_triples"] = processed_df["triples"].apply(
    lambda x: [map_to_ontology(triple) for triple in x]
)

processed_df.to_csv("enhanced_triples.csv", index=False)

print(processed_df[["title", "enhanced_triples"]])


TypeError: 'bool' object is not subscriptable